In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader, StorageContext, load_index_from_storage
documents=SimpleDirectoryReader("data").load_data()

In [3]:
index=VectorStoreIndex.from_documents(documents,show_progress=True)
index.storage_context.persist(persist_dir="index")

c:\Users\JayVo\anaconda3\envs\chest_xray\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 54/54 [00:03<00:00, 16.99it/s]


In [4]:
storage_context = StorageContext.from_defaults(persist_dir="index")
index = load_index_from_storage(storage_context)

In [4]:
retriever = index.as_retriever(similarity_top_k=6)

In [9]:
nodes = retriever.retrieve("What powers does the prime minister have")

In [6]:
for node in nodes:
    print(node.text)



AUSTRALIA’S CONSTITUTIONivDespite the structure of the Constitution there is no strict demarcation between the legislative and executive 
powers of the Commonwealth. Only the Parliament can pass Acts, but these Acts often confer on the 
Executive Government the power to make regulations, rules and by-laws in relation to matters relevant to the 
particular Acts.
For example, the Parliament may enact in the Customs Act that no person may bring a ‘prohibited import’ into 
Australia and then leave it to the Executive to specify in the Customs Regulations what is a ‘prohibited import’. 
This delegation of legislative power is not as extreme as it may appear, however, as both Houses of Parliament 
usually retain the power to ‘disallow’ (that is, reject), within a specified time, any regulation which has been 
made by the Executive.
The distinction between the Parliament and the Executive Government is further blurred by the fact that the 
Prime Minister and the other Government Ministers (wh

In [11]:
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
reranker = SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=5)
reranked_nodes = reranker.postprocess_nodes(nodes=nodes, query_str="What powers does the prime minister have")

In [118]:
combined_context = "\n".join(node.text for node in reranked_nodes)

In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

In [138]:
rewriting_llm = ChatOpenAI(
	temperature=0,
	model_name="gpt-3.5-turbo"
)

final_llm = ChatOpenAI(
	temperature=0,
	model_name="gpt-3.5-turbo"
)

# Initialize memory
memory = ConversationBufferMemory()


In [110]:
#Set Initial Prompt
rewriting_prompt_str = """
[INST] 
Given the Chat History and Question, rephrase the question so it can be a standalone question.  I will give some examples.

```
Example 1: If no chat history is present, then just return the original question
Chat History: 
Question: What is the power of the prime minister?
Your response: What is the power of the prime minister?
Note: You are returning  "What is the power of the prime minister?", not "Rephrased Question:  What is the power of the prime minister?"
```

```
Example 2: 
Chat History: 
Human: What is the power of the prime minister?
AI: The prime minister has many powers
Question: What else can he do?
Your response: What else can the prime minister do? 
Note: You are returning What else can the prime minister do?, not Rephrased Question: What else can the prime minister do?
```

With those examples, here are the actual chat history and input questions:
Chat History: {chat_history}
Question: {question}
[/INST] 
"""
rewriting_prompt = ChatPromptTemplate.from_template(rewriting_prompt_str)

In [137]:

response = rewriting_llm.invoke(rewriting_prompt.invoke({"chat_history":"Human: Who is Chris? \nAI: Your mom", "question":"What does he do?"}).to_string()).content

In [128]:
# Set final prompt
final_prompt_str = """
[INST] 
Given the following chat history and context, answer the following question.

Context: {context}
Chat History: {chat_history}
Question: {question}
[/INST] 
"""
final_prompt = ChatPromptTemplate.from_template(final_prompt_str)


# Update history

In [129]:
final_llm.invoke(final_prompt.invoke({"context":combined_context, "chat_history":memory.chat_memory.__str__(), "question":"What are his powers?"}).to_string())

AIMessage(content='The Prime Minister of Australia has the power to lead the government, make decisions on behalf of the government, represent Australia internationally, and appoint and dismiss government ministers. The Prime Minister is also responsible for setting government policy, managing the cabinet, and overseeing the administration of government departments.', response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 3563, 'total_tokens': 3618}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8a4d73f4-9511-4937-af80-79239cd056c0-0', usage_metadata={'input_tokens': 3563, 'output_tokens': 55, 'total_tokens': 3618})

In [125]:
memory.save_context({"HumanMessage":"Who is the prime minister"},{"AIMessage":"Yeah"})

In [126]:
memory.chat_memory.__str__()

'Human: Who is the prime minister\nAI: Yeah'